# Find the best location to open a fast food restaurant in New York
Couresa Data Science Course Final Assignment - Capstone Project
**************

In [1]:
# import libs
import numpy as np
import pandas as pd

In [2]:
#load the 
neighborhood_nta = pd.read_csv('./nynta.csv')

neighborhood_nta.head()

,BoroCode,the_geom,CountyFIPS,BoroName,NTACode,NTAName,Shape_Leng,Shape_Area
0,3,MULTIPOLYGON (((-73.97604935657381 40.63127590...,47,Brooklyn,BK88,Borough Park,"39,247.2278309","54,005,019.048"
1,4,MULTIPOLYGON (((-73.80379022888246 40.77561011...,81,Queens,QN51,Murray Hill,"33,266.9048721","52,488,277.5915"
2,4,MULTIPOLYGON (((-73.8610972440186 40.763664477...,81,Queens,QN27,East Elmhurst,"19,816.7121316","19,726,845.5925"
3,3,MULTIPOLYGON (((-73.94826499590911 40.63860718...,47,Brooklyn,BK95,Erasmus,"18,756.7018646","14,522,603.5915"
4,4,MULTIPOLYGON (((-73.75725671509139 40.71813860...,81,Queens,QN07,Hollis,"20,976.3355996","22,887,772.8087"


In [3]:
#calculate the longitude and latitude of a neighborhood base on the points of polygon of that neighborhood
def cal_neighborhood_coor(geom):
    geom = geom.replace('MULTIPOLYGON ','')
    geom = geom.replace('(','')
    geom = geom.replace(')','')
    
    parsed_array = str.split(geom, sep = ',')
    
    point_list = []
    for it in parsed_array:
        it = str.strip(it)
        point_list.append(str.split(it, sep = ' ' ))
    
    point_list = np.array(point_list)
    point_list = point_list.astype(np.float)
    
    return np.mean(point_list, axis = 0)

#create columns for longitude and latitude for each neighborhood.
def set_neighborhood_coor(row):
    
    coor_info = cal_neighborhood_coor(row['the_geom'])
    
    row['Longitude'] = coor_info[0]
    row['Latitude'] = coor_info[1]
    return row

    
neighborhood_nta = neighborhood_nta.apply(set_neighborhood_coor  , axis = 1 )

neighborhood_nta = neighborhood_nta[['NTACode','NTAName','BoroName','Longitude','Latitude']]
neighborhood_nta['NTAName'] = neighborhood_nta['NTAName'].apply(lambda name : name.upper())
neighborhood_nta['BoroName'] = neighborhood_nta['BoroName'].apply(lambda name : name.upper())
neighborhood_nta.head()

,NTACode,NTAName,BoroName,Longitude,Latitude
0,BK88,BOROUGH PARK,BROOKLYN,-73.987896,40.630659
1,QN51,MURRAY HILL,QUEENS,-73.807671,40.768094
2,QN27,EAST ELMHURST,QUEENS,-73.866046,40.763459
3,BK95,ERASMUS,BROOKLYN,-73.952030,40.645548
4,QN07,HOLLIS,QUEENS,-73.764067,40.710497


In [40]:
neighborhood_sales = pd.read_csv("./medianAskingRent_OneBd.csv")

neighborhood_sales = neighborhood_sales[neighborhood_sales['areaType'] == 'neighborhood' ]
neighborhood_sales['Borough'] = neighborhood_sales['Borough'].apply(lambda name : name.upper())
neighborhood_sales['areaName'] = neighborhood_sales['areaName'].apply(lambda name : name.upper())

neighborhood_sales['avg_rent']=neighborhood_sales[['2019-11','2019-12','2020-01','2020-02','2020-03', '2020-04', '2020-05', '2020-06', '2020-07', '2020-08', '2020-09', '2020-10']].mean(axis=1)
neighborhood_sales = neighborhood_sales[['areaName','Borough', 'avg_rent']]

neighborhood_sales.head()

,areaName,Borough,avg_rent
5,ASTORIA,QUEENS,1954.833333
6,AUBURNDALE,QUEENS,1600.000000
7,BATH BEACH,BROOKLYN,1647.571429
8,BATTERY PARK CITY,MANHATTAN,3679.916667
9,BAY RIDGE,BROOKLYN,1726.750000


In [43]:
merged_neigh_info = pd.merge(neighborhood_nta, neighborhood_sales, how='left', left_on='NTAName', right_on='areaName')
merged_neigh_info.head(10)
merged_neigh_info[ merged_neigh_info["areaName"].isnull()]


,NTACode,NTAName,BoroName,Longitude,Latitude,areaName,Borough,avg_rent
1,QN51,MURRAY HILL,QUEENS,-73.807671,40.768094,NaN,NaN,NaN
3,BK95,ERASMUS,BROOKLYN,-73.952030,40.645548,NaN,NaN,NaN
5,BK25,HOMECREST,BROOKLYN,-73.964715,40.598374,NaN,NaN,NaN
6,QN41,FRESH MEADOWS-UTOPIA,QUEENS,-73.785526,40.734983,NaN,NaN,NaN
12,BK93,STARRETT CITY,BROOKLYN,-73.884302,40.644918,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
189,BX63,WEST CONCOURSE,BRONX,-73.923685,40.830902,NaN,NaN,NaN
191,QN50,ELMHURST-MASPETH,QUEENS,-73.892844,40.738241,NaN,NaN,NaN
192,SI24,TODT HILL-EMERSON HILL-HEARTLAND VILLAGE-LIGHT...,STATEN ISLAND,-74.128289,40.588726,NaN,NaN,NaN
193,SI36,OLD TOWN-DONGAN HILLS-SOUTH BEACH,STATEN ISLAND,-74.079600,40.584197,NaN,NaN,NaN


Download from https://streeteasy.com/blog/data-dashboard/?agg=Total&metric=Inventory&type=Rentals&bedrooms=Any%20Bedrooms&property=Any%20Property%20Type&minDate=2010-01-01&maxDate=2020-10-01&area=Flatiron,Brooklyn%20Heights